In [1]:
import numpy as np

In [13]:
class MF():
    
    def __init__(self,R,K,alpha,beta,iterations):
        """
            R (ndarray) : user-item rating matrix
            K (int) : Latent factors
            alpha (float) : learning rate
            beta (float) : regularization parameter
        """
        
        self.R = R
        self.num_users,self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
    
    
    def train(self):
        #Initialize user and item latent vectors
        self.P = np.random.normal(scale=1/self.K,size=(self.num_users,self.K))
        self.Q = np.random.normal(scale=1/self.K,size=(self.num_items,self.K))
        
        #Initialize Biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(R!=0)])
        
        #Create Training sample which is a sparse matrix
        self.samples = [(i,j,self.R[i,j])
                   for i in range(self.num_users)
                   for j in range(self.num_items)
                   if self.R[i,j]>0
                  ]
        
        #Perform Stochastic Gradient Descent
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i,mse))
            if (i+1)%10 == 0:
                print('Iteration:%d ; error:%0.4f' %(i+1,mse))
            
        return training_process
                    
    
    def mse(self):
        """
        Function to calculate Mean Square Error
        """
        xs,ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x,y in zip(xs,ys):
            error+=pow(self.R[x,y] - predicted[x,y],2)
            
        return np.sqrt(error)
    
    def sgd(self):
        """
            Perform Stochastic Gradient Descent
        """
        for i,j,r in self.samples:
            #Compute Error and Prediction
            prediction = self.get_rating(i,j)
            error = (r - prediction)
            
            #Update Bias
            self.b_u[i]+= self.alpha * (error - self.beta * self.b_u[i])
            self.b_i[j]+= self.alpha * (error - self.beta * self.b_i[j])
            
            #Update Latent Matrices P and Q
            self.P[i,:]+= self.alpha * (error * self.Q[j,:] - self.beta * self.P[i,:])
            self.Q[j,:]+= self.alpha * (error * self.P[i,:] - self.beta * self.Q[j,:])
            
         
     
    def get_rating(self,i,j):
        """
            Get predicted rating of user i and item j
        """
        prediction = self.b_u[i] + self.b_i[j] + self.b + self.P[i,:].dot(self.Q[j,:].T)
        return prediction
    
    def full_matrix(self):
        """Compute the Rating matrix"""
        
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

In [14]:
R = np.array([
    [5, 3, 0, 1],
    [4, 0, 0, 1],
    [1, 1, 0, 5],
    [1, 0, 0, 4],
    [0, 1, 5, 4],
])

mf = MF(R,K=2,alpha=0.1,beta=0.01,iterations=25)
mf.train()

Iteration:10 ; error:0.0780
Iteration:20 ; error:0.0372


[(0, 5.6252992146366267),
 (1, 4.7877092765070008),
 (2, 3.5299894153313773),
 (3, 1.8146001652806407),
 (4, 0.76785277423130571),
 (5, 0.41304112850601948),
 (6, 0.24568422002001697),
 (7, 0.15909113791402676),
 (8, 0.10955399570012507),
 (9, 0.077992173866949413),
 (10, 0.058476405671732463),
 (11, 0.050908853604228331),
 (12, 0.042738187798533347),
 (13, 0.042374080240417986),
 (14, 0.040674294699036127),
 (15, 0.03824168637302975),
 (16, 0.036739605762285195),
 (17, 0.038264782163726974),
 (18, 0.038259066823439582),
 (19, 0.037168019950246031),
 (20, 0.038063045703234162),
 (21, 0.036883967924191519),
 (22, 0.039836159445385246),
 (23, 0.034460407655242262),
 (24, 0.032447497334925149)]

In [15]:
mf.full_matrix()

array([[ 4.98439652,  3.00438503,  2.91979122,  1.00781268],
       [ 3.99930287,  2.45040009,  3.68909348,  1.01210146],
       [ 1.00640453,  1.00589212,  4.04657535,  4.98976884],
       [ 1.00871459,  0.80704971,  3.69277202,  3.99488641],
       [ 0.99670578,  1.01376523,  4.98950309,  4.00005134]])